# Student Services MCP Demo Notebook

Author: Eidan J. Rosado

This notebook is meant as an example to end-users on how to leverage the StudentServicesMCPClient from the repo and use the exposed tools. For details on how to use this with Cursor, see README.md.

### Import Dependencies and Constants

In [3]:
# First cell - Import the MCP session
import os
import json
import pandas as pd
from src.client import StudentServicesMCPClient
from src.utils.generate_test_jwt import generate_jwt
from src.config.settings import settings

# Choose between your hugging face space or local server
BASE_URL = f"http://{settings.server.HOST}:{settings.server.PORT}/mcp"
HF_SPACE_URL = f"{settings.hf.HUGGINGFACE_SPACE_URL}/mcp"

Note: Due to the way HuggingFace spaces work, the private space will allow the first call with the token to go through, but each call following that call requires the Bearer token and that is not currently supported in the library's current state.

In [4]:
# _, token = generate_jwt()  # Use to test JWT flow

### Connect to Server

In [5]:
# Setup the client
client = StudentServicesMCPClient(
    base_url=HF_SPACE_URL,
    auth_token=settings.hf.HUGGINGFACE_TOKEN,
)
await client.connect_to_server()


Connecting to server at: https://edyvision-student-services-demo-mcp.hf.space/mcp

Connected to server with tools: ['root__get', 'fetch_student_profile', 'fetch_students', 'fetch_academic_history', 'check_financial_aid_eligibility']


### Fetch Top n Students

In [6]:
# Fetch students
limit = 10
students_result = await client.fetch_students(limit=limit)

students = json.loads(students_result[0].text)
students_df = pd.DataFrame(students)
students_df

,student_id,name,courses,is_need_based_qualified,enrollment_status,major,program,year,gpa
0,df62674f-5641-4657-a614-901a22ea76f2,Allison Hill,"['ENG202', 'BIO204', 'PHIL101', 'CS101', 'HIST...",True,enrolled,English Literature,B.A. in Comparative Literature,Senior,2.22
1,d777f2b6-906f-4703-a5f3-efac47766ac0,Megan Mcclain,"['ENG202', 'BIO204', 'HIST300']",True,enrolled,Nursing,B.S. in Nursing (BSN),Junior,2.84
2,1daf22b9-fbdc-48d8-ac64-abb8d4b775b0,Allen Robinson,"['ENG202', 'CHEM121', 'HIST300', 'PSY110', 'CS...",False,enrolled,Nursing,B.S. in Nursing (BSN),Junior,2.05
3,c0ad1251-fef1-4ec0-8914-046fb4119492,Cristian Santos,"['ENG202', 'PHIL101', 'PSY110', 'CS101']",True,enrolled,English Literature,B.A. in Creative Writing,Senior,2.85
4,e73992d2-6535-43d8-9c3e-afb3bb6463c8,Kevin Pacheco,"['HIST300', 'PHIL101', 'CS101', 'ENG202', 'MAT...",True,enrolled,Computer Science,B.S. in Cybersecurity,Senior,2.76
5,1ef1b24e-57d3-4bbc-ac6e-e7998526fb98,Melissa Peterson,"['ENG202', 'MATH150', 'CS101', 'PHIL101', 'BIO...",True,enrolled,Economics,B.S. in Quantitative Economics,Freshman,2.16
6,4c6549d0-096d-4c92-b271-911d93117d42,Gabrielle Davis,"['PHIL101', 'PSY110', 'ENG202']",False,enrolled,Biology,B.S. in Molecular Biology,Freshman,3.97
7,fed2a7af-7567-4ba8-b431-f9c86a87812f,Lindsey Roman,"['MATH150', 'BIO204', 'CHEM121', 'PSY110', 'CS...",False,enrolled,English Literature,B.A. in English Literature,Sophomore,3.67
8,5dbeccea-947a-4817-b3cb-2c9231bac4f5,Valerie Gray,"['PSY110', 'ENG202', 'PHIL101', 'CHEM121']",False,enrolled,Nursing,RN to BSN Track,Senior,3.27
9,47fcc672-72e7-4ef0-87d4-51e800c3dd31,Lisa Hensley,"['CS101', 'BIO204', 'CHEM121', 'PSY110', 'ENG2...",False,enrolled,English Literature,B.A. in English Literature,Senior,3.38


### Fetch Financial Aid Eligibility for Students

In [7]:
students[0]

{'student_id': 'df62674f-5641-4657-a614-901a22ea76f2',
 'name': 'Allison Hill',
 'courses': "['ENG202', 'BIO204', 'PHIL101', 'CS101', 'HIST300']",
 'is_need_based_qualified': True,
 'enrollment_status': 'enrolled',
 'major': 'English Literature',
 'program': 'B.A. in Comparative Literature',
 'year': 'Senior',
 'gpa': 2.22}

In [8]:
# Check eligibility for Allison Hill
eligibility = await client.check_financial_aid_eligibility(students[0]["student_id"])
print(eligibility[0].text)

"Student Allison Hill (ID: df62674f-5641-4657-a614-901a22ea76f2) has a GPA of 2.22 in English Literature. They are eligible for the ['Humanities Excellence Scholarship']. Requirements: ['Field of Study in Humanities']"


In [9]:
students[4]

{'student_id': 'e73992d2-6535-43d8-9c3e-afb3bb6463c8',
 'name': 'Kevin Pacheco',
 'courses': "['HIST300', 'PHIL101', 'CS101', 'ENG202', 'MATH150']",
 'is_need_based_qualified': True,
 'enrollment_status': 'enrolled',
 'major': 'Computer Science',
 'program': 'B.S. in Cybersecurity',
 'year': 'Senior',
 'gpa': 2.76}

In [10]:
# Check eligibility for Student 4
eligibility = await client.check_financial_aid_eligibility(students[4]["student_id"])
print(eligibility[0].text)

"Student Kevin Pacheco (ID: e73992d2-6535-43d8-9c3e-afb3bb6463c8) has a GPA of 2.76 in Computer Science. They are eligible for the ['STEM Excellence Award']. Requirements: ['Field of Study in STEM']"
